<a href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Cancer Detection

## Objective
* To build a dense prediction model
* To begin reading current papers in DNN research

## Deliverable
For this lab, you will turn in a notebook that describes your efforts at creating
a pytorch radiologist. Your final deliverable is a notebook that has (1) a deep network,
(2) method of calculating accuracy, 
(3) images that show the dense prediction produced by your network on the pos_test_000072.png image (index 172 in the validation dataset).
This is an image in the test set that your network will not have seen before.
This image, and the ground truth labeling, is shown below.
(And is contained in the downloadable dataset below).

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=200&tok=a8ac31&media=cs501r_f2016:pos_test_000072_output.png)
<img src="http://liftothers.org/dokuwiki/lib/exe/fetch.php?media=cs501r_f2016:pos_test_000072.png" width="200">


## Grading standards
Your notebook will be graded on the following:
* 40% Proper design, creation and debugging of a dense prediction network
* 20% Proper implementation of train/test set accuracy measure
* 20% Tidy visualizations of loss of your dense predictor during training
* 20% Test image output


## Data set
The data is given as a set of 1024×1024 PNG images. Each input image (in 
the ```inputs``` directory) is an RGB image of a section of tissue,
and there a file with the same name (in the ```outputs``` directory) 
that has a dense labeling of whether or not a section of tissue is cancerous
(white pixels mean “cancerous”, while black pixels mean “not cancerous”).

The data has been pre-split for you into test and training splits.
Filenames also reflect whether or not the image has any cancer at all 
(files starting with ```pos_``` have some cancerous pixels, while files 
starting with ```neg_``` have no cancer anywhere).
All of the data is hand-labeled, so the dataset is not very large.
That means that overfitting is a real possibility.

## Description
For a video including some tips and tricks that can help with this lab: [https://youtu.be/Ms19kgK_D8w](https://youtu.be/Ms19kgK_D8w)
For this lab, you will implement a virtual radiologist.
You are given images of possibly cancerous tissue samples, 
and you must build a detector that identifies where in the tissue cancer may reside.

---

### Part 0
Watch and follow video tutorial:

https://youtu.be/Ms19kgK_D8w

**TODO:**

* Watch tutorial

**DONE:**

### Part 1
Implement a dense predictor

In previous labs and lectures, we have talked about DNNs that classify an 
entire image as a single class. Here, however, we are interested in a more nuanced classification: 
given an input image, we would like to identify each pixel that is possibly cancerous. 
That means that instead of a single output, your network should output an “image”, 
where each output pixel of your network represents the probability that a pixel is cancerous.

___

Implement your network topology

Use the “Deep Convolution U-Net” from this paper: [(U-Net: Convolutional Networks for Biomedical Image Segmentation)](https://arxiv.org/pdf/1505.04597.pdf) 

![(Figure 1)](https://lh3.googleusercontent.com/qnHiB3B2KRxC3NjiSDtY08_DgDGTDsHcO6PP53oNRuct-p2QXCR-gyLkDveO850F2tTAhIOPC5Ha06NP9xq1JPsVAHlQ5UXA5V-9zkUrJHGhP_MNHFoRGnjBz1vn1p8P2rMWhlAb6HQ=w2400)

You should use existing pytorch functions (not your own Conv2D module), such as ```nn.Conv2d``` and ```nn.CrossEntropyLoss```;
you will also need the pytorch function ```torch.cat``` and ```nn.ConvTranspose2d```

```torch.cat``` allows you to concatenate tensors.
```nn.ConvTranspose2d``` is the opposite of ```nn.Conv2d```.
It is used to bring an image from low res to higher res.
[This blog](https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0) should help you understand this function in detail.

You can simplify the implementation of this lab by padding the feature maps as they pass through each convolution. This will make the concatenation process easier, though this is technically a departure from the cropping technique outlined in the orginal U-Net paper. 

Note that the simplest network you could implement (with all the desired properties)
is just a single convolution layer with two filters and no relu! 
Why is that? (of course it wouldn't work very well!)

**TODO:**

* Understand the U-Net architecture
* Understand concatenation of inputs from multiple prior layers
* Understand ConvTranspose
* Answer Question / Reflect on simplest network with the desired properties

**DONE:**


___
The intention of this lab is to learn how to make deep neural nets. We'll help you with the implementation of Dataset.
This code will download the dataset for you so that you are ready to use it and focus on network
implementation, losses and accuracies.

In [7]:
!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

# assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

gpu = torch.device('mps')
x = torch.randn(2, 2, device=gpu)

WARNING: You may run into an error that says "RuntimeError: CUDA out of memory."

In this case, the memory required for your batch is larger than what the GPU is capable of. You can solve this problem by adjusting the image size or the batch size and then restarting the runtime. 

In [7]:
class CancerDataset(Dataset):
  def __init__(self, root, download=True, size=512, train=True):
    if download and not os.path.exists(os.path.join(root, 'cancer_data')):
      datasets.utils.download_url('http://liftothers.org/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      self.extract_gzip(os.path.join(root, 'cancer_data.tar.gz'))
      self.extract_tar(os.path.join(root, 'cancer_data.tar'))
    
    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'cancer_data', 'cancer_data')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'inputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'outputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  @staticmethod
  def extract_gzip(gzip_path, remove_finished=False):
    print('Extracting {}'.format(gzip_path))
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())
    if remove_finished:
      os.unlink(gzip_path)
  
  @staticmethod
  def extract_tar(tar_path):
    print('Untarring {}'.format(tar_path))
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]
  
  def __len__(self):
    return len(self.dataset_folder)

In [9]:
# You'll probably want a function or something to test input / output sizes of the ConvTranspose2d layer
def size_of_convTrans2d(i, o, k, s, p):
  t = torch.randn(i, 28, 28)
  net = nn.ConvTranspose2d(in_channels=i, out_channels=o, kernel_size=k, stride=s, padding=p)

  output = net(t)

  return output.size()

i, o, k, s, p = 1024, 1024, 2, 2, 0 # note that increasing the stride multiplies the height and width by that number

size_of_convTrans2d(i, o, k, s, p)

torch.Size([1024, 56, 56])

In [10]:
# Since you will be using the output of one network in two places(convolution and maxpooling),
# you can't use nn.Sequential.
# Instead you will write up the network like normal variable assignment as the example shown below:
# You are welcome (and encouraged) to use the built-in batch normalization and dropout layer.

# TODO: You need to change this to fit the UNet structure!!!
class CancerDetection(nn.Module):
  def __init__(self): # CancerDetection constructor
    # You always need to use super to call nn.Module's init function when creating a class that inherits nn.Module.
    super(CancerDetection, self).__init__()
    
    # Always initialize your layers in the init function.
    self.conv1 = nn.Conv2d(3, 64, kernel_size = 3, stride = 1, padding = 1)
    self.relu1 = nn.ReLU()

    self.conv2 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1)
    self.relu2 = nn.ReLU()
    
    self.pool1 = nn.MaxPool2d(kernel_size = 2)

    self.conv3 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)
    self.relu3 = nn.ReLU()

    self.conv4 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1)
    self.relu4 = nn.ReLU()

    self.pool2 = nn.MaxPool2d(kernel_size = 2)

    self.conv5 = nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1)
    self.relu5 = nn.ReLU()

    self.conv6 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
    self.relu6 = nn.ReLU()

    self.pool3 = nn.MaxPool2d(kernel_size = 2)

    self.conv7 = nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding = 1)
    self.relu7 = nn.ReLU()

    self.conv8 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
    self.relu8 = nn.ReLU()

    self.pool4 = nn.MaxPool2d(kernel_size = 2)
    
    # this is now the lowest resolution; we begin upscaling beyond this point

    self.conv9 = nn.Conv2d(512, 1024, kernel_size = 3, stride = 1, padding = 1)
    self.relu9 = nn.ReLU()

    self.conv10 = nn.Conv2d(1024, 1024, kernel_size = 3, stride = 1, padding = 1)
    self.relu10 = nn.ReLU()

    self.trans1 = nn.ConvTranspose2d(in_channels = 1024, out_channels = 512, kernel_size = 2, stride = 2, padding = 0)

    self.conv11 = nn.Conv2d(1024, 512, kernel_size = 3, stride = 1, padding = 1)
    self.relu11 = nn.ReLU()

    self.conv12 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
    self.relu12 = nn.ReLU()

    self.trans2 = nn.ConvTranspose2d(in_channels = 512, out_channels = 256, kernel_size = 2, stride = 2, padding = 0)

    self.conv13 = nn.Conv2d(512, 256, kernel_size = 3, stride = 1, padding = 1)
    self.relu13 = nn.ReLU()

    self.conv14 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
    self.relu14 = nn.ReLU()

    self.trans3 = nn.ConvTranspose2d(in_channels = 256, out_channels = 128, kernel_size = 2, stride = 2, padding = 0)

    self.conv15 = nn.Conv2d(256, 128, kernel_size = 3, stride = 1, padding = 1)
    self.relu15 = nn.ReLU()

    self.conv16 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1)
    self.relu16 = nn.ReLU()

    self.trans4 = nn.ConvTranspose2d(in_channels = 128, out_channels = 64, kernel_size = 2, stride = 2, padding = 0)

    self.conv17 = nn.Conv2d(128, 64, kernel_size = 3, stride = 1, padding = 1)
    self.relu17 = nn.ReLU()

    self.conv18 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1)
    self.relu18 = nn.ReLU()

    self.conv_last = nn.Conv2d(64, 2, kernel_size = 1, stride = 1, padding = 0)

  def forward(self, input): # nn.Module sets up a hook that calls forward when you "call" the module object: net(x) calls net.forward(x)
    conv1_out = self.conv1(input)
    relu1_out = self.relu1(conv1_out)

    conv2_out = self.conv2(relu1_out)
    relu2_out = self.relu2(conv2_out)

    pool1_out = self.pool1(relu2_out)

    conv3_out = self.conv3(pool1_out)
    relu3_out = self.relu3(conv3_out)

    conv4_out = self.conv4(relu3_out)
    relu4_out = self.relu4(conv4_out)

    pool2_out = self.pool2(relu4_out)

    conv5_out = self.conv5(pool2_out)
    relu5_out = self.relu5(conv5_out)

    conv6_out = self.conv6(relu5_out)
    relu6_out = self.relu6(conv6_out)

    pool3_out = self.pool3(relu6_out)

    conv7_out = self.conv7(pool3_out)
    relu7_out = self.relu7(conv7_out)

    conv8_out = self.conv8(relu7_out)
    relu8_out = self.relu8(conv8_out)

    pool4_out = self.pool4(relu8_out)

    # begin upscaling

    conv9_out = self.conv9(pool4_out)
    relu9_out = self.relu9(conv9_out)

    conv10_out = self.conv10(relu9_out)
    relu10_out = self.relu10(conv10_out)

    trans1_out = self.trans1(relu10_out)
    
    concat4 = torch.cat((trans1_out, relu8_out), dim = 1)

    conv11_out = self.conv11(concat4)
    relu11_out = self.relu11(conv11_out)

    conv12_out = self.conv12(relu11_out)
    relu12_out = self.relu12(conv12_out)

    trans2_out = self.trans2(relu12_out)

    concat3 = torch.cat((trans2_out, relu6_out), dim = 1)

    conv13_out = self.conv13(concat3)
    relu13_out = self.relu13(conv13_out)

    conv14_out = self.conv14(relu13_out)
    relu14_out = self.relu14(conv14_out)

    trans3_out = self.trans3(relu14_out)

    concat2 = torch.cat((trans3_out, relu4_out), dim = 1) 

    conv15_out = self.conv15(concat2)
    relu15_out = self.relu15(conv15_out)

    conv16_out = self.conv16(relu15_out)
    relu16_out = self.relu16(conv16_out)

    trans4_out = self.trans4(relu16_out)

    concat1 = torch.cat((trans4_out, relu2_out), dim = 1)

    conv17_out = self.conv17(concat1)
    relu17_out = self.relu17(conv17_out)

    conv18_out = self.conv18(relu17_out)
    relu18_out = self.relu18(conv18_out)

    output = self.conv_last(relu18_out)
    
    return output


In [4]:
# Create your datasets, dataloaders and neural network as you have before
# Tip: Pass shuffle=True to constructor for training dataloader to automatically shuffle training data each epoch. You can shuffle the validation loader, but you don't need to.

In [11]:
# This is what was talked about in the video for memory management

EPOCHS = 1

losses = []
validation = []

accuracy = []
validation_accuracy = []

def scope():
  try:
    # your code for calling dataset and dataloader
    train_dataset = CancerDataset(root='/tmp/mycancerdata', size=400)
    val_dataset = CancerDataset(root='/tmp/mycancerdata', size=300, train=False)

    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, pin_memory=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True, pin_memory=True, num_workers=2)

    model = CancerDetection()
    model.cuda()

    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    gc.collect()
    print(torch.cuda.memory_allocated() / 1e9)
    
    for epoch in range(EPOCHS):
      loop = tqdm(total=len(train_loader), position=0, leave=False)
    
    # Call your model, figure out loss and accuracy
      for batch, (x, y_truth) in enumerate(train_loader):
        x, y_truth = x.cuda(non_blocking=True), y_truth.cuda(non_blocking=True)
        
        optimizer.zero_grad()

        y_hat = model(x)

        y_hat_layer1 = y_hat[:, 0, :, :]
        y_hat_layer2 = y_hat[:, 1, :, :]

        print("\n")
        print("y_hat: ", y_hat.size())
        print("y_truth: ", y_truth.size())
        
        loss = objective(y_hat, y_truth)
        loss.backward()
        losses.append(loss.item())

        acc = (y_hat.argmax(1) == y_truth).float().mean()
        accuracy.append(acc.item())

        mem = torch.cuda.memory_allocated() / 1e9

        loop.set_description('epoch:{} loss:{:.4f} accuracy:{:.3f} memory:{:.3f}'.format(epoch, loss, acc, mem))
        loop.udate(1)

        optimizer.step()

        if batch % 500 == 0:
          val = np.mean( [objective(model(x.cuda()), y.cuda()).item() for x, y in val_loader] )
          validation.append(len(losses), val)

          val_acc = np.mean( [(model(x.cuda()).argmax(1) == y.cuda()).float().mean().item() for x, y in val_loader ] )
          validation_accuracy.append(len(accuracy), val_acc)

  except:
    __ITB__()
    
scope()

  0%|          | 0/2750494655 [00:00<?, ?it/s]

Extracting /tmp/mycancerdata/cancer_data.tar.gz
Untarring /tmp/mycancerdata/cancer_data.tar
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py in cuda(self=CancerDetection(
  (conv1): Conv2d(3, 64, kernel...onv2d(64, 2, kernel_size=(1, 1), stride=(1, 1))
), device=None)
    747             Module: self
    748         """
--> 749         return self._apply(lambda t: t.cuda(device))
        self._apply = <bound method Module._apply of CancerDetection(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (conv


___

### Part 2

Plot performance over time

Please generate two plots:

 One that shows loss on the training and validation set as a function of training time. 

 One that shows accuracy on the training and validation set as a function of training time. 

 Make sure your axes are labeled!

**TODO:**

* Plot training/validation loss as function of training time (not epochs)
* Plot training/validation accuracy as function of training time (not epochs)

**DONE:**



In [ ]:
# Your plotting code here


**NOTE:**

Guessing that the pixel is not cancerous every single time will give you an accuracy of ~ 85%.
Your trained network should be able to do better than that (but you will not be graded on accuracy).
This is the result I got after 1 hour or training.

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=d23e0b&media=cs501r_f2016:training_accuracy.png)
![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=bb8e3c&media=cs501r_f2016:training_loss.png)

___

### Part 3

Generate at least 5 predictions on the pos_test_000072.png image and display them as images. These predictions should be made at a reasonable interval (e.g. every epoch). 

You can load this image from the file pos_test_000072.png, or you can get it from the dataset object. It is item 172 of the validation dataset.
You can print both the data instance (x) and the ground-truth label (y_hat) to see how well your network predicts on that instance.

To do this, calculate the output of your trained network on the pos_test_000072.png image,
then make a hard decision (cancerous/not-cancerous) for each pixel.
The resulting image should be black-and-white, where white pixels represent areas
the network considers cancerous.

**TODO:**

**DONE:**



In [ ]:
# Code for testing prediction on an image
